In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F


# Load Silver data (replace table name if different)
silver_df = spark.table("Covid_Silver")

display(silver_df.limit(5))


StatementMeta(, 633e42e8-f2bf-480c-bbb9-f9c56a7e9b2b, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cb6e7c27-d7b8-47ee-80e8-80b66913a7f0)

In [2]:
# Global metrics
global_df = silver_df.agg(
    F.sum("cases").alias("total_cases"),
    F.sum("deaths").alias("total_deaths"),
    F.sum("recovered").alias("total_recovered"),
    F.sum("tests").alias("total_tests"),
    F.avg("death_rate_pct").alias("avg_death_rate_pct"),
    F.avg("tests_per_thousand").alias("avg_tests_per_thousand")
)

display(global_df)


StatementMeta(, 633e42e8-f2bf-480c-bbb9-f9c56a7e9b2b, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8b148767-6990-4a55-bad4-fa9d6b8a0719)

### **Cell 3 – Top 10 Countries by Cases**



In [3]:
top_cases_df = (
    silver_df
    .select("country", "cases", "deaths", "death_rate_pct")
    .orderBy(F.col("cases").desc())
    .limit(10)
)

display(top_cases_df)


StatementMeta(, 633e42e8-f2bf-480c-bbb9-f9c56a7e9b2b, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5cbecdc6-3904-410a-bba3-30c6639c090c)

### **Cell 4 – Top 10 Countries by Death Rate**

In [4]:
top_death_rate_df = (
    silver_df
    .select("country", "cases", "deaths", "death_rate_pct")
    .filter("cases > 10000")  # filter small sample countries
    .orderBy(F.col("death_rate_pct").desc())
    .limit(10)
)

display(top_death_rate_df)

StatementMeta(, 633e42e8-f2bf-480c-bbb9-f9c56a7e9b2b, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6946c28d-8fc1-4a2c-99d6-c09e3f07baf3)

In [5]:
# Save final gold data (country-level KPIs)
gold_df = silver_df.select(
    "country",
    "cases",
    "deaths",
    "recovered",
    "tests",
    "population",
    "death_rate_pct",
    "tests_per_thousand",
    "cases_per_million"
)

# Write as Gold table
gold_df.write.mode("overwrite").saveAsTable("Covid_Gold")

print("Gold Layer table created: Covid_Gold")


StatementMeta(, 633e42e8-f2bf-480c-bbb9-f9c56a7e9b2b, 7, Finished, Available, Finished)

Gold Layer table created: Covid_Gold
